In [ ]:
import pandas as pd
import numpy as np
import puffin.tardis_importer as ti

In [2]:
ti.batch_download(exchange='binance-futures', symbol_list=['BTCUSDT'], start_date='2021-10-14', end_date = '2021-10-15', data_types =['quotes', 'trades'])

In [3]:
ti_trades = ti.generate_trade_data_from_gzip('BTCUSDT', './datasets/binance-futures_trades_2021-10-14_BTCUSDT.csv.gz')
ti_trades.to_csv('./datasets/binance-futures_trades_2021-10-14_BTCUSDT.csv', index = False)

ti_tob = ti.generate_tob_data_from_gzip('BTCUSDT', './datasets/binance-futures_quotes_2021-10-14_BTCUSDT.csv.gz')
ti_tob.to_csv('./datasets/binance-futures_quotes_2021-10-14_BTCUSDT.csv', index = False)

In [16]:
df_tob = pd.read_parquet('/Users/rahmanw/Downloads/Sandbox/20211014_4627_tob.parquet')
df_trades = pd.read_parquet('/Users/rahmanw/Downloads/Sandbox/20211014_4627_trade.parquet')
print(len(df_trades))

1550910


In [20]:
#reduce the intersection to just where the trade id's overlap (we collect a bit of extra date on either side of midnight)
first_id = ti_trades['trade_id'].iloc[0]
last_id = ti_trades['trade_id'].iloc[-1]
print(first_id, last_id)
df_trades = df_trades[(df_trades['ignore_first_trade_id'] >= first_id) & (df_trades['ignore_last_trade_id'] <= last_id)]
print(len(df_trades))

1536569832 1539609453
1548314


In [18]:
df_trades

,trade_date_nanos,instrument_id,receive_timestamp_nanos,exchange_timestamp_nanos,price,qty,buyer_is_market_maker,ignore_first_trade_id,ignore_last_trade_id
2594,1634169600000000000,4627,1634169608820374457,1634169602475000000,57393.65,0.005,True,1536569832,1536569832
2595,1634169600000000000,4627,1634169611487457032,1634169602972000000,57393.65,1.027,True,1536569833,1536569846
2596,1634169600000000000,4627,1634169611646244133,1634169602990000000,57393.66,0.198,False,1536569847,1536569856
2597,1634169600000000000,4627,1634169611678412326,1634169603078000000,57393.65,0.141,True,1536569857,1536569857
2598,1634169600000000000,4627,1634169611688295673,1634169603093000000,57393.66,1.156,False,1536569858,1536569862
...,...,...,...,...,...,...,...,...,...
1550903,1634169600000000000,4627,1634255999362236704,1634255999204000000,57379.86,0.001,False,1539609445,1539609445
1550904,1634169600000000000,4627,1634255999642112559,1634255999481000000,57379.86,0.021,False,1539609446,1539609446
1550905,1634169600000000000,4627,1634255999909614965,1634255999899000000,57379.85,0.009,True,1539609447,1539609450
1550906,1634169600000000000,4627,1634255999955239842,1634255999899000000,57377.57,0.071,True,1539609451,1539609451


In [19]:
ti_trades

,instrument_id,receive_timestamp_nanos,exchange_timestamp_nanos,price,qty,buyer_is_market_maker,trade_id
0,4627,1634169608672425984,1634169602475000064,57393.65,0.005,True,1536569832
1,4627,1634169611430328064,1634169602972000000,57393.65,0.001,True,1536569833
2,4627,1634169611466356992,1634169602976000000,57393.65,0.050,True,1536569834
3,4627,1634169611472185088,1634169602982000128,57393.65,0.185,True,1536569835
4,4627,1634169611472370944,1634169602982000128,57393.65,0.002,True,1536569836
...,...,...,...,...,...,...,...
3039500,4627,1634255999912271104,1634255999899000064,57379.85,0.002,True,1539609449
3039501,4627,1634255999912512000,1634255999899000064,57379.85,0.002,True,1539609450
3039502,4627,1634255999913005056,1634255999899000064,57377.57,0.071,True,1539609451
3039503,4627,1634255999960958976,1634255999947000064,57378.20,0.082,True,1539609452


In [ ]:
agg_prices = []
agg_qty = []
for i in range(len(df_trades)):
    start_id = df_trades.iloc[i]['ignore_first_trade_id']
    end_id = df_trades.iloc[i]['ignore_last_trade_id']
    prices = ti_trades['price'][(ti_trades['trade_id'] >= start_id) & (ti_trades['trade_id'] <= end_id)].values
    agg_prices.append(prices)
    agg_qty = ti_trades['qty'][(ti_trades['trade_id'] >= start_id) & (ti_trades['trade_id'] <= end_id)].sum()
    agg_qty = float('{:,.3f}'.format(agg_qty))
    print(i, agg_qty, np.unique(prices))

0 0.005 [57393.65]
1 1.027 [57393.65]
2 0.198 [57393.66]
3 0.141 [57393.65]
4 1.156 [57393.66]
5 0.175 [57394.91]
6 0.002 [57393.66]
7 0.571 [57393.65]
8 0.006 [57393.66]
9 0.07 [57393.67]
10 0.147 [57393.65]
11 0.002 [57393.65]
12 0.007 [57393.66]
13 0.174 [57393.67]
14 0.024 [57393.66]
15 0.241 [57393.65]
16 0.174 [57393.67]
17 0.101 [57393.65]
18 0.157 [57396.12]
19 0.154 [57396.65]
20 0.058 [57396.66]
21 0.001 [57393.66]
22 0.001 [57393.67]
23 0.002 [57393.65]
24 0.001 [57393.66]
25 0.003 [57394.12]
26 1.102 [57393.65]
27 0.681 [57393.66]
28 0.172 [57394.12]
29 0.005 [57396.2]
30 0.062 [57393.66]
31 0.793 [57393.66]
32 0.003 [57396.2]
33 0.005 [57396.2]
34 0.007 [57393.66]
35 0.174 [57393.67]
36 0.018 [57393.67]
37 0.592 [57393.66]
38 0.234 [57393.66]
39 0.269 [57393.65]
40 0.065 [57393.66]
41 0.338 [57393.15]
42 0.001 [57392.48]
43 0.001 [57391.65]
44 0.002 [57393.66]
45 0.01 [57393.65]
46 0.097 [57391.65]
47 0.341 [57390.71]
48 0.095 [57389.78]
49 0.001 [57393.66]
50 0.003 [57394

405 0.09 [57417.76]
406 0.061 [57417.5]
407 0.104 [57417.45]
408 0.022 [57416.93]
409 0.131 [57416.92]
410 1.28 [57416.91]
411 0.345 [57416.61]
412 0.394 [57416.6]
413 0.312 [57419.2]
414 0.392 [57416.6]
415 0.103 [57416.49]
416 0.262 [57416.34]
417 0.112 [57416.1]
418 0.184 [57415.92]
419 0.126 [57415.62]
420 0.2 [57415.59]
421 0.181 [57415.58]
422 0.08 [57414.58]
423 0.001 [57415.73]
424 0.08 [57415.42]
425 0.009 [57413.32]
426 0.16 [57413.24]
427 0.16 [57412.77]
428 0.002 [57412.71]
429 0.16 [57412.39]
430 0.2 [57415.64]
431 0.008 [57415.72]
432 0.08 [57415.59]
433 0.003 [57412.05]
434 0.541 [57410.39]
435 0.328 [57410.21]
436 0.261 [57410.4]
437 0.007 [57410.22]
438 0.172 [57410.21]
439 0.157 [57410.41]
440 0.004 [57407.48]
441 0.001 [57406.11]
442 0.004 [57405.46]
443 0.448 [57405.45]
444 0.066 [57405.06]
445 0.01 [57404.76]
446 0.21 [57404.75]
447 0.003 [57404.75]
448 0.024 [57404.76]
449 0.428 [57404.77]
450 0.001 [57404.9]
451 0.006 [57404.91]
452 0.001 [57409.11]
453 0.022 [57

821 0.005 [57402.62]
822 0.104 [57398.17]
823 0.002 [57398.02]
824 0.018 [57396.64]
825 0.002 [57396.64]
826 0.087 [57396.63]
827 0.002 [57395.73]
828 0.004 [57395.72]
829 0.007 [57392.06]
830 0.004 [57392.06]
831 0.006 [57392.05]
832 0.022 [57392.06]
833 0.002 [57392.05]
834 0.004 [57392.06]
835 0.012 [57392.05]
836 0.006 [57392.06]
837 0.002 [57392.06]
838 0.008 [57392.06]
839 0.033 [57392.06]
840 0.087 [57392.07]
841 0.009 [57392.1]
842 0.019 [57394.07]
843 0.003 [57394.08]
844 0.222 [57394.07]
845 0.002 [57394.08]
846 0.138 [57394.08]
847 0.023 [57394.09]
848 0.002 [57394.09]
849 0.023 [57394.1]
850 0.001 [57394.11]
851 0.001 [57399.14]
852 0.001 [57399.13]
853 0.258 [57399.14]
854 0.055 [57401.14]
855 0.01 [57401.15]
856 0.002 [57401.15]
857 0.076 [57401.15]
858 0.001 [57401.14]
859 0.002 [57401.15]
860 0.001 [57401.14]
861 0.073 [57401.15]
862 0.041 [57401.15]
863 0.045 [57401.19]
864 0.001 [57401.2]
865 0.002 [57401.2]
866 0.011 [57401.19]
867 0.007 [57401.19]
868 0.017 [57401.1

1229 0.658 [57403.51]
1230 0.174 [57403.52]
1231 0.534 [57403.78]
1232 0.003 [57405.46]
1233 0.003 [57408.28]
1234 0.001 [57409.5]
1235 0.009 [57409.97]
1236 0.001 [57413.28]
1237 0.001 [57415.29]
1238 0.087 [57415.3]
1239 0.002 [57418.79]
1240 0.011 [57418.8]
1241 0.044 [57415.29]
1242 0.034 [57415.28]
1243 0.022 [57415.28]
1244 0.054 [57415.28]
1245 0.005 [57415.28]
1246 0.002 [57415.29]
1247 0.008 [57415.28]
1248 0.414 [57415.29]
1249 0.01 [57415.28]
1250 0.003 [57415.29]
1251 0.016 [57415.8]
1252 0.003 [57416.13]
1253 0.011 [57416.12]
1254 0.01 [57416.13]
1255 0.001 [57416.12]
1256 0.011 [57416.13]
1257 0.017 [57416.12]
1258 0.025 [57416.13]
1259 0.08 [57416.14]
1260 0.03 [57417.56]
1261 0.008 [57419.45]
1262 0.007 [57419.46]
1263 0.046 [57419.46]
1264 0.001 [57419.46]
1265 0.002 [57419.45]
1266 0.005 [57419.46]
1267 0.06 [57419.45]
1268 0.017 [57419.46]
1269 0.033 [57419.46]
1270 0.034 [57424.17]
1271 0.034 [57424.2]
1272 0.087 [57419.47]
1273 0.004 [57424.16]
1274 0.004 [57424.19

1621 0.003 [57443.2]
1622 0.066 [57446.59]
1623 0.002 [57446.6]
1624 0.001 [57448.45]
1625 0.039 [57449.69]
1626 0.085 [57449.99]
1627 0.002 [57449.99]
1628 0.002 [57450.]
1629 0.017 [57450.]
1630 0.001 [57449.99]
1631 0.048 [57450.]
1632 0.005 [57449.99]
1633 0.004 [57450.]
1634 0.001 [57450.]
1635 0.061 [57450.]
1636 0.008 [57450.]
1637 0.331 [57449.99]
1638 0.035 [57448.32]
1639 0.002 [57446.89]
1640 0.092 [57446.86]
1641 0.087 [57446.59]
1642 0.076 [57445.97]
1643 0.241 [57444.37]
1644 0.099 [57444.36]
1645 0.099 [57444.21]
1646 0.099 [57443.95]
1647 0.099 [57443.38]
1648 0.174 [57443.36]
1649 0.099 [57443.21]
1650 0.175 [57443.2]
1651 0.175 [57443.19]
1652 0.003 [57442.9]
1653 0.001 [57442.46]
1654 0.174 [57442.14]
1655 0.024 [57441.68]
1656 0.003 [57440.9]
1657 0.003 [57440.65]
1658 0.038 [57440.58]
1659 0.099 [57440.02]
1660 0.075 [57438.9]
1661 0.002 [57441.69]
1662 0.022 [57438.47]
1663 0.175 [57437.77]
1664 0.06 [57437.13]
1665 0.001 [57436.6]
1666 0.012 [57435.55]
1667 0.007

...

64534 0.001 [57684.42]
64535 0.002 [57684.99]
64536 0.001 [57685.]
64537 0.159 [57685.08]
64538 0.979 [57685.09]
64539 0.004 [57685.11]
64540 0.004 [57685.21]
64541 0.023 [57685.3]
64542 0.004 [57685.31]
64543 0.003 [57685.48]
64544 1.225 [57685.49]
64545 1.215 [57685.5]
64546 0.004 [57685.51]
64547 0.001 [57685.91]
64548 0.033 [57686.02]
64549 0.1 [57686.16]
64550 0.004 [57686.31]
64551 0.001 [57686.85]
64552 0.86 [57687.]
64553 0.3 [57687.02]
64554 0.705 [57687.03]
64555 0.022 [57687.4]
64556 0.001 [57687.83]
64557 0.2 [57687.91]
64558 0.033 [57688.]
64559 0.008 [57688.13]
64560 0.001 [57688.69]
64561 0.916 [57689.18]
64562 2.0 [57689.19]
64563 0.004 [57689.2]
64564 0.001 [57689.7]
64565 0.007 [57690.]
64566 0.603 [57690.09]
64567 1.077 [57690.84]
64568 1.0 [57690.85]
64569 0.197 [57691.02]
64570 0.162 [57691.03]
64571 0.697 [57691.25]
64572 0.07 [57691.81]
64573 0.165 [57691.87]
64574 0.347 [57691.95]
64575 0.586 [57691.96]
64576 0.002 [57692.28]
64577 0.001 [57692.62]
64578 0.174 [

64926 0.005 [57731.93]
64927 0.574 [57731.94]
64928 0.087 [57731.95]
64929 0.008 [57732.72]
64930 0.001 [57732.75]
64931 0.076 [57732.74]
64932 0.002 [57732.75]
64933 0.055 [57732.74]
64934 0.148 [57731.93]
64935 0.009 [57731.94]
64936 0.136 [57731.93]
64937 0.002 [57731.94]
64938 0.048 [57731.93]
64939 0.001 [57731.94]
64940 0.047 [57731.93]
64941 0.087 [57731.59]
64942 0.033 [57728.84]
64943 0.003 [57728.85]
64944 0.018 [57728.84]
64945 0.001 [57728.85]
64946 0.036 [57728.84]
64947 1.255 [57725.26]
64948 0.087 [57725.27]
64949 0.023 [57728.83]
64950 0.315 [57728.84]
64951 0.222 [57725.25]
64952 1.485 [57725.26]
64953 0.087 [57725.27]
64954 0.004 [57728.65]
64955 0.208 [57728.66]
64956 0.383 [57728.79]
64957 0.1 [57728.8]
64958 0.001 [57728.82]
64959 0.178 [57725.26]
64960 0.1 [57725.25]
64961 0.001 [57724.99]
64962 0.038 [57722.99]
64963 0.098 [57722.99]
64964 0.001 [57720.68]
64965 0.11 [57719.25]
64966 0.001 [57717.69]
64967 0.04 [57716.92]
64968 0.003 [57716.91]
64969 0.001 [57715

65290 0.034 [57734.05]
65291 0.018 [57733.92]
65292 0.217 [57733.91]
65293 0.005 [57734.86]
65294 0.018 [57733.91]
65295 0.068 [57734.1]
65296 0.007 [57733.9]
65297 0.001 [57732.12]
65298 0.001 [57731.55]
65299 0.278 [57731.54]
65300 0.001 [57731.48]
65301 0.081 [57730.62]
65302 0.107 [57730.6]
65303 0.009 [57730.61]
65304 0.4 [57730.6]
65305 0.003 [57730.34]
65306 0.004 [57729.14]
65307 0.044 [57728.84]
65308 0.039 [57728.33]
65309 0.087 [57728.3]
65310 0.139 [57727.76]
65311 0.003 [57727.43]
65312 0.006 [57727.23]
65313 0.067 [57726.17]
65314 0.018 [57728.1]
65315 0.003 [57728.57]
65316 0.017 [57730.26]
65317 0.015 [57730.35]
65318 0.001 [57728.]
65319 0.003 [57725.22]
65320 0.001 [57728.01]
65321 0.003 [57725.15]
65322 0.007 [57725.14]
65323 0.004 [57725.15]
65324 0.433 [57725.14]
65325 0.323 [57724.96]
65326 0.24 [57724.88]
65327 0.001 [57725.15]
65328 0.004 [57725.14]
65329 0.013 [57724.88]
65330 0.001 [57724.88]
65331 0.002 [57724.89]
65332 0.017 [57723.]
65333 0.118 [57722.99]
6

65682 0.199 [57682.29]
65683 0.035 [57682.3]
65684 0.048 [57686.45]
65685 0.08 [57686.47]
65686 0.001 [57688.]
65687 0.087 [57688.01]
65688 0.104 [57690.56]
65689 0.014 [57691.83]
65690 0.001 [57692.11]
65691 2.411 [57692.11]
65692 0.316 [57692.1]
65693 0.104 [57691.83]
65694 0.342 [57691.82]
65695 0.061 [57689.88]
65696 0.09 [57689.87]
65697 0.292 [57689.31]
65698 3.03 [57689.3]
65699 0.004 [57689.]
65700 0.182 [57688.01]
65701 2.065 [57688.]
65702 0.387 [57692.12]
65703 0.072 [57692.11]
65704 0.114 [57691.73]
65705 0.194 [57690.78]
65706 0.347 [57690.79]
65707 0.213 [57690.78]
65708 0.087 [57690.49]
65709 0.003 [57684.02]
65710 0.004 [57682.31]
65711 0.024 [57682.3]
65712 0.016 [57682.3]
65713 0.05 [57682.3]
65714 0.01 [57682.31]
65715 0.039 [57682.31]
65716 0.394 [57682.3]
65717 0.002 [57682.29]
65718 0.104 [57682.19]
65719 0.002 [57681.37]
65720 0.089 [57681.36]
65721 0.017 [57681.37]
65722 0.087 [57678.8]
65723 0.318 [57677.11]
65724 0.094 [57677.1]
65725 0.001 [57677.11]
65726 0.

66070 0.001 [57742.23]
66071 0.004 [57742.31]
66072 0.286 [57742.33]
66073 0.174 [57739.33]
66074 0.061 [57742.33]
66075 0.023 [57742.37]
66076 0.004 [57742.51]
66077 0.533 [57742.61]
66078 0.133 [57733.9]
66079 0.568 [57742.61]
66080 0.001 [57742.9]
66081 0.006 [57743.]
66082 0.004 [57743.04]
66083 0.004 [57743.08]
66084 0.004 [57743.31]
66085 0.705 [57743.75]
66086 0.166 [57733.51]
66087 0.234 [57743.75]
66088 0.06 [57733.07]
66089 0.096 [57743.75]
66090 0.166 [57733.9]
66091 0.571 [57743.75]
66092 0.052 [57738.72]
66093 0.601 [57743.75]
66094 0.423 [57736.77]
66095 2.871 [57743.75]
66096 0.001 [57744.]
66097 0.569 [57744.68]
66098 1.291 [57744.99]
66099 0.008 [57745.]
66100 0.197 [57745.32]
66101 0.001 [57745.52]
66102 0.001 [57745.55]
66103 0.001 [57745.68]
66104 0.001 [57745.75]
66105 0.001 [57745.86]
66106 0.002 [57745.87]
66107 0.002 [57745.99]
66108 0.001 [57746.05]
66109 0.07 [57746.08]
66110 0.005 [57746.17]
66111 0.003 [57746.24]
66112 0.004 [57746.26]
66113 0.014 [57746.39]